In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
%pip install spuco
%pip install wilds

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=b2d57907e153b3ca06e2353ced1e0d48986f763210b00d6bed9bcfbafbfba6b3
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [ ]:
%pip install -q pytorch-metric-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from spuco.datasets import GroupLabeledDatasetWrapper, WILDSDatasetWrapper, SpuCoBirds
from wilds import get_dataset
import torchvision.transforms as transforms
import cv2
import torchvision.models as models
from pytorch_metric_learning import losses
import torch.optim as optim
from torch.optim import lr_scheduler

device = torch.device("cuda")

In [ ]:
class Config:
  label_noise = 0.001
  size = 32
  flip_probability = 0.5
  batch_size = 128
  backbone_pretrained = False
  projection_dim = 128
  T_max = 10
  lr = 5e-4
  min_lr = 1e-5
  weight_decay = 1e-6
  temperature = 0.5

## Dataset

In [ ]:
# # SpuCoBirds can't automatically download the dataset since the
# # file got renamed from spuco_birds.tar.gz to spuco_birds_v1.tar.gz.
# # The dataset needs to be manually loaded from your local drive.
# # Link: https://ucla.box.com/shared/static/zrsx09ik7lqad1e389w06f1c4nbajo9z

# trainset = SpuCoBirds(
#     root="/content/drive/MyDrive/",
#     download=False,
#     split="train",
#     label_noise=Config.label_noise
# )
# trainset.initialize()

# valset = SpuCoBirds(
#     root="/content/drive/MyDrive/",
#     download=False,
#     split="val"
# )
# valset.initialize()

In [ ]:
## Augmentation
from torchvision.transforms import v2

# Convert to float
transform_img = transforms.Compose([
            v2.Resize(256),
            v2.CenterCrop(224),
            v2.ToTensor(),
            v2.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

transform = transforms.Compose([
    v2.RandomResizedCrop(Config.size, scale=(0.08, 1.0), ratio=(0.75, 1.25)),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    v2.RandomGrayscale(p=0.2),
    v2.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [ ]:
dataset = get_dataset(dataset="waterbirds", download=True, root_dir='/data/waterbirds')

# Get the subsets
train_data = dataset.get_subset("train", transform=transform_img)
val_data = dataset.get_subset("val", frac=1, transform=transform_img)

# Wrap to make SpuCo compatible
trainset = WILDSDatasetWrapper(dataset=train_data, metadata_spurious_label="background", verbose=True)
valset = WILDSDatasetWrapper(dataset=val_data, metadata_spurious_label="background", verbose=True)
print('\n')
print(len(trainset))
print(len(valset))

You can also download the dataset manually at https://wilds.stanford.edu/downloads.


489291776Byte [01:37, 5043544.42Byte/s]


Extracting /data/waterbirds/waterbirds_v1.0/archive.tar.gz to /data/waterbirds/waterbirds_v1.0

It took 1.75 minutes to download and uncompress the dataset.



Partitioning data indices into groups: 100%|██████████| 1199/1199 [00:00<00:00, 600904.59it/s]



4795
1199


In [ ]:
class SpucoDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img = self.dataset[index][0]
        label = self.dataset[index][1]

        if self.transform is None:
          return img, label

        augmented_img = transform(img)
        return (img, augmented_img), label

In [ ]:
train_dataset = SpucoDataset(trainset, transform)
val_dataset = SpucoDataset(valset)

dataloaders = {'train': DataLoader(dataset=train_dataset, batch_size=Config.batch_size, num_workers=2, pin_memory=True, shuffle=True),
         'eval': DataLoader(dataset=val_dataset, batch_size=16, num_workers=2, pin_memory=True, shuffle=False)}

In [ ]:
# print(train_dataset.dataset[0][0])
# train_dataset.dataset[0][0].shape
len(dataloaders['train'])

38

## Contrastive Loss

In [ ]:
## https://kevinmusgrave.github.io/pytorch-metric-learning/losses/#ntxentloss
## https://kevinmusgrave.github.io/pytorch-metric-learning/losses/#selfsupervisedloss
criterion = losses.NTXentLoss(temperature=Config.temperature)

## Model

In [ ]:
## https://github.com/berfukaraca/SSL-Cifar10-ContrastiveLearning/blob/main/colab_notebooks/SSL_SimCLR_Cifar10_CL.ipynb

class SimCLR_model(nn.Module):
    def __init__(self, base_encoder, projection_dim, n_features):
        super(SimCLR_model, self).__init__()
        self.n_features = n_features
        self.base_encoder = base_encoder

        self.projectionHead = nn.Sequential(
            nn.Linear(self.n_features, self.n_features, bias=False),
            nn.BatchNorm1d(self.n_features),
            nn.ReLU(), #non-linearity
            nn.Linear(self.n_features, projection_dim, bias=False),
            nn.BatchNorm1d(projection_dim),
        )

    def forward(self, x_i):
        out = self.projectionHead(self.base_encoder(x_i))

        return out

In [ ]:
## model and optimizer
model = models.resnet50()
# print(model)
clr_model = SimCLR_model(model, Config.projection_dim, 1000) ## output dimension of resnet18 is 1000
optimizer = optim.Adam(clr_model.parameters(), lr=Config.lr, weight_decay=Config.weight_decay)
## cosine annealing lr: https://arxiv.org/pdf/1608.03983.pdf
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.T_max, eta_min=Config.min_lr)

device = 'cpu'
clr_model.to(device)

SimCLR_model(
  (base_encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(


In [ ]:
torch.cuda.empty_cache()

In [ ]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    clr_model.train()
    s=0
    tot_loss = 0
    total = 0
    batch_idx=0
    i=0
    batch_num =1
    for batch_idx, (images, labels) in enumerate(dataloaders['train']):
        batch_tot_loss = 0
        if (batch_num % 50 == 1):
          print("in batch", batch_num)
        batch_num=batch_num+1
        images, augmented_images = images[0], images[1]
        images, augmented_images = images.to(device), augmented_images.to(device)

        optimizer.zero_grad()

        images_emb = clr_model(images)
        #augmented_images_emb = clr_model(augmented_images)

        #embs = torch.cat((images_emb, augmented_images_emb))
        #labels = torch.cat((labels, labels))

        loss = criterion(images_emb, labels)

        loss.backward()

        optimizer.step()
        scheduler.step

        tot_loss += loss.item()
        batch_tot_loss += loss.item()
        if (batch_num % 50 == 1):
          print('batch loss=', batch_tot_loss)

    print("epoch loss=",tot_loss)

    if epoch%20 == 0:
        modelstate_save_name = 'waterbird_modelstate_bs_restart_r.pt'
        path_modelstate = F"/content/drive/MyDrive/CS260D_final_project/model_parameters/{modelstate_save_name}"
        torch.save(clr_model.state_dict(), path_modelstate)

        parameters_end = {
            'net': clr_model.state_dict(),
            'epoch': epoch,
            'loss': tot_loss
      }
        paramsend_save_name = 'waterbird_paramsend_bs_restart_r.pt'
        path_paramsend = F"/content/drive/MyDrive/CS260D_final_project/model_parameters/{paramsend_save_name}"
        torch.save(parameters_end, path_paramsend)

start_epoch = 0
max_epoch = 60

for epoch in range(start_epoch, start_epoch+max_epoch):
  train(epoch)


Epoch: 0


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
epoch loss= 111.93056416511536

Epoch: 1


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
epoch loss= 102.12161111831665

Epoch: 2


...

in batch 1
epoch loss= 39.07579988241196

Epoch: 58


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
epoch loss= 34.93909966945648

Epoch: 59


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
epoch loss= 34.77257776260376


In [ ]:
# Load the pre-trained model parameters
paramsend_save_name = 'waterbird_paramsend_bs_restart_r.pt'
path_paramsend = "/content/drive/MyDrive/CS260D_final_project/model_parameters/waterbird_paramsend_bs_restart_r.pt"
# checkpoint = torch.load(path_paramsend)
checkpoint = torch.load(path_paramsend, map_location=torch.device('cpu'))
loss= checkpoint['loss']
epoch = checkpoint['epoch']

start_epoch=epoch+1
clr_model.load_state_dict(checkpoint['net'])

<All keys matched successfully>

## PCA and 2D Scatter Plot

In [ ]:
## https://www.kaggle.com/code/minc33/visualizing-high-dimensional-clusters

import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

torch.cuda.empty_cache()
# device = 'cpu'
clr_model.to(device)
tmp = []
embeddings = []
targets = []

for batch_idx, (images, labels) in enumerate(dataloaders['eval']):
  print(batch_idx)
  tmp.append(clr_model(images.to(device)).detach().cpu())
  targets.append(labels)
  torch.cuda.empty_cache()

# clr_model = None

embeddings = torch.cat(tmp).detach().cpu().numpy()
targets = torch.cat(targets).cpu().numpy()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74


In [ ]:
len(valset)

1199

In [ ]:
import pandas as pd

comp = 8
pca = PCA(n_components=comp)
pca_emb = pca.fit_transform(embeddings)
PCs = pd.DataFrame(pca_emb)
PCs.columns = [f"PC{i+ 1}" for i in range(comp)]
embs = pd.concat([pd.DataFrame(embeddings),PCs], axis=1, join='inner')

In [ ]:
pca.singular_values_

array([385.66995  ,  50.46564  ,  20.813457 ,   3.9054172,   2.6583967,
         1.7497642,   1.4339342,   1.2107346], dtype=float32)

In [ ]:
# Add labels to embedding df
embs['true_label'] = targets
for (true, spur), idxs in valset.group_partition.items():
  for i in idxs:
    embs.loc[i, 'spurious_label'] = spur
embs['spurious_label'] = embs['spurious_label'].astype(int)

In [ ]:
clusters = {k : np.stack([PCs.loc[i].values for i in v]) for (k, v) in valset.group_partition.items()}

In [ ]:
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# traces = []

# # for (k, v) in sorted(clusters.items()):
# #   traces.append(go.Scatter(
# #                     x = v[:, 0],
# #                     y = v[:, 1],
# #                     mode = "markers",
# #                     name = "True label {}, Subgroup {}".format(k[0], k[1]),
# #                     text = None))

# for true_label, df in embs.groupby(['true_label']):
#   traces.append(go.Scatter(x = df['PC1'],
#                            y = df['PC2'],
#                            mode = "markers",
#                            name = f"True label {true_label}",
#                            text = None,
#                            marker = dict(size = 4)
#                            )
#               )

<ipython-input-24-ae22b7a3578c>:15: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.



In [ ]:
traces = []
for true_label, df in embs.groupby(['true_label']):
  traces.append(go.Scatter(x = df['PC1'],
                           y = df['PC2'],
                           mode = "markers",
                           name = f"True label {true_label}",
                           text = None,
                           marker = dict(size = 4)
                           )
  )
title = "PCA analysis of Validation Data by True Label"
layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False),
              height = 1000,
              width = 1000
             )

fig = dict(data = traces, layout = layout)
iplot(fig)
go.Figure.write_html(fig,"/content/drive/MyDrive/CS260D_final_project/plots/waterbirds_PCA_true_plot.html") # write as html or image

<ipython-input-63-c11191799a5d>:2: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.



In [ ]:
traces = []
for spurious_label, df in embs.groupby(['spurious_label']):
  traces.append(go.Scatter(x = df['PC1'],
                           y = df['PC2'],
                           mode = "markers",
                           name = f"Spurious label {spurious_label}",
                           text = None,
                           marker = dict(size = 4)
                           )
  )
title = "PCA analysis of Validation Data by Spurious Label"
layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False),
              height = 1000,
              width = 1000
             )

fig = dict(data = traces, layout = layout)
iplot(fig)
go.Figure.write_html(fig,"/content/drive/MyDrive/CS260D_final_project/plots/waterbirds_PCA_spurious_plot.html") # write as html or image

<ipython-input-60-e824a9695bd9>:2: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.



## t-SNE

In [ ]:
from sklearn.manifold import TSNE

perplexity = 30
iteration = 3000

# #T-SNE with two dimensions
# tsne_2d = TSNE(n_components=2, perplexity=perplexity, n_iter=iteration)

# TCs_2d = pd.DataFrame(tsne_2d.fit_transform(embeddings))
# TCs_2d.columns = ["TC1_2d","TC2_2d"]
# tsne_embs = pd.concat([pd.DataFrame(embeddings),TCs_2d], axis=1, join='inner')

In [ ]:
# tsne_clusters = {k : np.stack([TCs_2d.loc[i].values for i in v]) for (k, v) in valset.group_partition.items()}

In [ ]:
# tsne_traces = []

# for (k, v) in sorted(tsne_clusters.items()):
#   tsne_traces.append(go.Scatter(
#                     x = v[:, 0],
#                     y = v[:, 1],
#                     mode = "markers",
#                     name = "True label {}, Subgroup {}".format(k[0], k[1]),
#                     text = None))

In [ ]:
# data = tsne_traces

# title = "Visualizing Clusters in Two Dimensions Using t-SNE"

# layout = dict(title = title,
#               xaxis= dict(title= 'TC1',ticklen= 5,zeroline= False),
#               yaxis= dict(title= 'TC2',ticklen= 5,zeroline= False)
#              )

# fig = dict(data = data, layout = layout)

# iplot(fig)

In [ ]:
go.Figure.write_html(fig,"/content/drive/MyDrive/CS260D_final_project/plots/waterbirds_tSNE_plot.html") # write as html or image

In [ ]:
#T-SNE with two dimensions
tsne = TSNE(n_components=2, perplexity=perplexity, n_iter = 3000)

TCs = pd.DataFrame(tsne.fit_transform(embeddings))
TCs.columns = ["TC1","TC2"]
TCs['true_label'] = embs['true_label']
TCs['spurious_label'] = embs['spurious_label']
# Find best perplexity value
traces = []
for true_label, df in TCs.groupby(['true_label']):
  traces.append(go.Scatter(x = df['TC1'], y = df['TC2'], mode = "markers", name = f"True label {true_label}", text = None))

title = f"t-SNE Cluster Visualization of Embedded Data Grouped by True Label, Perplexity = {perplexity}"
layout = dict(title = title,
              xaxis= dict(title= 'TC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'TC2',ticklen= 5,zeroline= False),
              width = 1000,
              height = 1000
              #zaxis= dict(title= 'TC3',ticklen= 5,zeroline= False)
            )

fig = dict(
    data = traces,
    layout = layout
)
iplot(fig)
go.Figure.write_html(fig,"/content/drive/MyDrive/CS260D_final_project/plots/waterbirds_tSNE_true_plot.html") # write as html or image

<ipython-input-65-58c2be87bbcc>:10: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.



In [ ]:
#T-SNE with two dimensions
tsne = TSNE(n_components=2, perplexity=perplexity, n_iter = 3000)

TCs = pd.DataFrame(tsne.fit_transform(embeddings))
TCs.columns = ["TC1","TC2"]
TCs['true_label'] = embs['true_label']
TCs['spurious_label'] = embs['spurious_label']
# Find best perplexity value
traces = []
for spurious_label, df in TCs.groupby(['spurious_label']):
  traces.append(go.Scatter(x = df['TC1'], y = df['TC2'], mode = "markers", name = f"Spurious label {spurious_label}", text = None))

title = f"t-SNE Cluster Visualization of Embedded Data Grouped by Spurious Label, Perplexity = {perplexity}"
layout = dict(title = title,
              xaxis= dict(title= 'TC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'TC2',ticklen= 5,zeroline= False),
              width = 1000,
              height = 1000
              #zaxis= dict(title= 'TC3',ticklen= 5,zeroline= False)
            )

fig = dict(
    data = traces,
    layout = layout
)
iplot(fig)
go.Figure.write_html(fig,"/content/drive/MyDrive/CS260D_final_project/plots/waterbirds_tSNE_spurious_plot.html") # write as html or image

<ipython-input-66-92f5e170e1f7>:10: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.



## PCA 3D plot

In [ ]:
import pandas as pd

pca_3d = PCA(n_components=3)
PCs_3d = pd.DataFrame(pca_3d.fit_transform(embeddings))
PCs_3d.columns = ["PC1_3d", "PC2_2d", "PC3_3D"]
pca_embs = pd.concat([pd.DataFrame(embeddings),PCs_3d], axis=1, join='inner')

clusters = {k : np.stack([PCs_3d.loc[i].values for i in v]) for (k, v) in valset.group_partition.items()}

traces = []

for (k, v) in sorted(clusters.items()):
  traces.append(go.Scatter3d(
                    x = v[:, 0],
                    y = v[:, 1],
                    z = v[:, 2],
                    mode = "markers",
                    name = "True label {}, Subgroup {}".format(k[0], k[1]),
                    text = None))

data = traces

title = "Visualizing Clusters in Three Dimensions Using PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

In [ ]:
go.Figure.write_html(fig,"/content/drive/MyDrive/CS260D_final_project/plots/waterbirds_PCA3D_plot.html") # write as html or image

## t-SNE 3D

In [ ]:
from sklearn.manifold import TSNE

perplexity = 50

#T-SNE with two dimensions
tsne_3d = TSNE(n_components=3, perplexity=perplexity)

TCs_3d = pd.DataFrame(tsne_3d.fit_transform(embeddings))
TCs_3d.columns = ["TC1_3d","TC3_2d","TC3_3d"]
tsne_embs = pd.concat([pd.DataFrame(embeddings),TCs_2d], axis=1, join='inner')

tsne_clusters = {k : np.stack([TCs_3d.loc[i].values for i in v]) for (k, v) in valset.group_partition.items()}

tsne_3d_traces = []

for (k, v) in sorted(tsne_clusters.items()):
  tsne_3d_traces.append(go.Scatter3d(
                    x = v[:, 0],
                    y = v[:, 1],
                    z = v[:, 2],
                    mode = "markers",
                    name = "True label {}, Subgroup {}".format(k[0], k[1]),
                    text = None))

data = tsne_3d_traces

title = "Visualizing Clusters in Two Dimensions Using t-SNE"

layout = dict(title = title,
              xaxis= dict(title= 'TC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'TC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

In [ ]:
go.Figure.write_html(fig,"/content/drive/MyDrive/CS260D_final_project/plots/waterbirds_tSNE3D_plot.html") # write as html or image

In [ ]:
T.ToPILImage()(train_dataset.__getitem__(0)[0][0])

In [ ]:
print(train_dataset.__getitem__(1)[1])

In [ ]:
T.ToPILImage()(train_dataset.__getitem__(1)[0][1])

In [ ]:
T.ToPILImage()(train_dataset.__getitem__(1)[0][0])

In [ ]:
T.ToPILImage()(trainset[700][0]).resize((28,28))

In [ ]:
T.ToPILImage()(trainset[11990][0]).resize((28,28))

In [ ]:
trainset.labels[11990]

In [ ]:
trainset.group_partition[(1,0)]